先从最基本的数学计算代码段的加速开始，实现在cpu和gpu上的无痛加速计算。
numba主要能够针对含有大量*循环结构*和*numpy*操作的代码进行加速

In [2]:
import numpy as np

1. 首先使用求素数的纯python代码

In [24]:
def prime_number_pure_python(end):
    prime_numbers = []
    for i in range(1, end):
        for j in range(2, np.sqrt(i).astype(int) + 1):
            if i % j == 0:
                break
            else:
                prime_numbers.append(i)
    return prime_numbers

In [85]:
%timeit -n 1 -r 5 prime_number_pure_python(1_000_000)

13.1 s ± 1.96 s per loop (mean ± std. dev. of 5 runs, 1 loop each)


2. 使用`numba`进行CPU上的加速，主要体现在以下几个方面
    + 使用完全避免python控制台运行的编译方法`@numba.njit`或`@numba.jit(nopython=True)`
    + 在数学计算中使用优化计算的属性`fastmath=True`
    + 使用避免`GIL`大锁的并行编译属性`parallel=True`和`nogil=True`
    + 在运行时**动态**设置运行的最大进程数`numba.set_num_threads(<cores>)`，其中`<cores>`为**逻辑处理器数量**
    + 在可以并行的循环中将`range()`更换为`numba.prange()`
    + 如果涉及`numpy.linalg`线性代数计算，则环境中需要安装`scipy`

In [72]:
import numba
from numba import prange

@numba.jit(nopython=True,
           fastmath=True,
           parallel=True,
           nogil=True)
def prime_number_cpu(end):
    prime_numbers = np.zeros((end))
    for i in prange(1, end):
        for j in prange(2, int(np.sqrt(i)) + 1):
            if i % j == 0:
                break
            else:
                prime_numbers[i-1] = i
    return prime_numbers

numba.set_num_threads(4)

In [89]:
%timeit -n 1 -r 5 prime_number_cpu(1_000_000)

1.59 s ± 298 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


4